In [1]:
import math
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import pandas as pd
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# 미디어 파이프 설정 변수 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True,min_detection_confidence=0.3, model_complexity=0)
mp_drawing = mp.solutions.drawing_utils


In [3]:
# 포즈 감지 함수
def detectPose(image,pose,display=True):
    # input 이미지 복사
    output_image = image.copy()
    # RGB로 변환
    imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    # 렌드마크 찍기
    results = pose.process(imageRGB)
    # 사진 사이즈 구하기
    height, width, _ = image.shape
    # 렌드마크 값 리스트 만들기
    landmarks = []

    if results.pose_landmarks:
        # 렌드마크 그리기
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
        #렌드마크 정규값 landmarks 리스트에 넣기
        for landmark in results.pose_landmarks.landmark:

            landmarks.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))
    # # 사진 결과 보기
    # if display:
    #     plt.figure(figsize=[22,22])
    #     plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
    #     plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    #     # 3차원으로도 봐보기
    #     mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
    # else:
    
    return output_image, landmarks

In [4]:
# 각도 구하는 함수
def calculateAngle(landmark1, landmark2, landmark3):
    x1,y1,_ = landmark1 # z 좌표는 사용 x
    x2,y2,_ = landmark2
    x3,y3,_ = landmark3
    # 3점의 각도 -> 선 2개 -> 각도
    angle = math.degrees(math.atan2(y3-y2, x3-x2) - math.atan2(y1-y2,x1-x2))

    angle = np.abs(angle)
    # 음수의 각도가 나오면 양수로 바꾸기
    if angle >180.0:
        angle = 360 - angle
        
    return angle

In [16]:
# 사진 한장 -> 3가지 각도를 저장할 프레임 선언 
Datas = pd.DataFrame()

In [6]:
# 사진 파일 경로 설정
right_path = './legraise/right/'
right_pic = os.listdir('./legraise/right/')
leg_path = './legraise/leg/'
leg_pic = os.listdir('./legraise/leg/')
headup_path = './legraise/headup/'
headup_pic = os.listdir('./legraise/headup/')

In [20]:
# 개수 카운트
counter = 0
# 처음 상태
stage = 'start'
a = [] #시작각도
b = [] #고점
c = [] #1회 끝 (다시 시작 각도)

df = pd.DataFrame()

for i in range(int(len(leg_pic))):
    # right_pic, headup_pic, leg_pic 바꿔주기 -> 라벨달기 편하게
    image = cv2.imread(leg_path+leg_pic[i])
    output_image, landmarks = detectPose(image,pose,display=False)

    if landmarks:
        # 랜드마크 추출 후 각도 계산
        # 귀 - 어깨 - 엉덩이 => 고개 각도 구하기 위함
        left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_EAR.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])
        
        # 어깨 - 엉덩이 - 무릎 => 올리는 동작 각도 구하기 위함 -> Main angle
        angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
        
        # 엉덩이 - 무릎 - 발목 => 무릎 꺾였는지 보기 위함
        left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                        landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
       
       # start -> 각각 리스트 길이가 초기에 0이고 1회 완료시 a,c 의 리스트 길이가 같아짐
        if len(a) == 0 or len(a) == len(c) :
            if angle >140 : 
                print('다리 쭉 펴진 상태 -> 140~180도')
                # 이때의 고개 , 무릎 각도 저장
                a.append(left_knee_angle)
                a.append(left_shoulder_angle)
                stage='start'
        
        
        # 고점에 도착했을때            
        elif len(b) < len(a) :
            if angle < 110 :
                print('다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기')
                b.append(left_knee_angle)
                b.append(left_shoulder_angle)
                stage = 'up'
        
        # 1회 완성 했을 때
        elif len(c) < len(b) :
            if angle >140 :
                print('다시 다시 내린 상태 -> 전과같이 140~180도')
                c.append(left_knee_angle)
                c.append(left_shoulder_angle)
                stage = 'complete'
                counter += 1 # 1회 동작 완료
                # 동작 완료 후 a,b,c 에 있는 각도 -> 한개의 행으로 합치기 (한개 할때의 각각의 각도가 저장됨)
                globals()[f'f{counter}'] = a+b+c
                print(globals()[f'f{counter}'])
                # 임시 dataframe에 저장
                df= df.append([globals()[f'f{counter}']])
                # 다시 abc 리스트 0으로 초기화
                a = []
                b = []
                c = []

        # # 횟수 카운트 하는 사각형 만들기
        # cv2.rectangle(output_image, (0,0), (225,73), (245,117,16), -1)

        # # 횟수
        # cv2.putText(output_image, '1set', (15,12), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        # cv2.putText(output_image, str(counter), 
        #             (10,60), 
        #         cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # # 상태
        # cv2.putText(output_image, 'STAGE', (65,12), 
        #                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        # cv2.putText(output_image, stage, 
        #                 (60,60), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # plt.figure(figsize=[10,10])
        # plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');


다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기
다시 다시 내린 상태 -> 전과같이 140~180도
[87.10500198126117, 145.1755108430432, 111.80140948635182, 147.55356907357756, 117.30890486184887, 142.57840360781344]
다리 쭉 펴진 상태 -> 140~180도


C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\1222695273.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append([globals()[f'f{counter}']])


다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기
다시 다시 내린 상태 -> 전과같이 140~180도
[117.63285225945035, 144.53054768127288, 113.70288093537948, 146.36824247436465, 114.37021014880204, 144.668569623565]
다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기
다시 다시 내린 상태 -> 전과같이 140~180도
[113.02350096595933, 144.517421350721, 120.73101758982628, 151.99918056307152, 113.21419818951195, 144.77267805324067]
다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기
다시 다시 내린 상태 -> 전과같이 140~180도
[173.66231466769236, 159.69815656829732, 138.1814889278133, 139.99411679406035, 85.93538436006442, 130.20986927763678]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\1222695273.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append([globals()[f'f{counter}']])


다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기
다시 다시 내린 상태 -> 전과같이 140~180도
[81.80748396735801, 131.17600478235943, 112.82012400353591, 155.03398624893683, 63.21058629984949, 147.66105393596234]
다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기


...

C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\1222695273.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append([globals()[f'f{counter}']])


다시 다시 내린 상태 -> 전과같이 140~180도
[164.0516006246735, 143.49100669345606, 126.5475279536602, 145.04701854305586, 172.42576963339357, 142.62671788260224]
다리 쭉 펴진 상태 -> 140~180도
다리 들어 올린상태 -> 110도보다 작아질 정도로 올리기


C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\1222695273.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append([globals()[f'f{counter}']])


다시 다시 내린 상태 -> 전과같이 140~180도
[178.77415023300867, 140.67741666204532, 104.8255577587168, 146.75867507580978, 179.59319072990974, 145.6398369517957]
다리 쭉 펴진 상태 -> 140~180도


C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\1222695273.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= df.append([globals()[f'f{counter}']])


In [21]:
# 라벨값 추가 -> 0 : 정상 , 1 : 목 , 2 : 다리
df['y'] = 2
# 임시 데이터 프레임을 차례차례 넣어주기
Datas = Datas.append(df)
Datas

C:\Users\smhrd\AppData\Local\Temp\ipykernel_14320\310810813.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Datas = Datas.append(df)


,0,1,2,3,4,5,y
0,170.457588,151.629708,164.946441,152.001762,169.258602,150.901133,0
0,169.258602,150.901133,160.183737,151.013425,175.739815,152.158787,0
0,173.679040,151.482938,157.068102,149.034440,157.542638,141.082513,0
0,145.664882,142.793593,158.368783,152.322292,151.356825,142.188505,0
0,142.979015,145.097972,156.726610,151.590351,149.836584,147.106296,0
...,...,...,...,...,...,...,...
0,48.476051,149.596980,101.858638,147.595457,46.460233,147.636279,2
0,179.292681,144.212256,111.146213,138.776554,116.696013,136.210231,2
0,176.793335,142.626921,110.144832,139.187621,178.019533,144.323592,2
0,164.051601,143.491007,126.547528,145.047019,172.425770,142.626718,2


In [22]:
# 3동작 했으면 csv 파일로 저장하기
Datas.to_csv("Datas.csv",index=None)